In [21]:
# Install dependencies
!pip install mediapipe flask flask-cors pyngrok sentence-transformers joblib

# Flask and CORS
from flask import Flask, request, jsonify
from flask_cors import CORS

# Ngrok (for tunneling in Colab/Notebook)
from pyngrok import ngrok, conf
conf.get_default().auth_token = "Your_Authentication_Token"

# Core libraries
import os
import cv2
import numpy as np
import pickle
import joblib

# MediaPipe (for pose/landmark detection)
import mediapipe as mp

# Deep learning model loading
from tensorflow.keras.models import load_model

# Sentence transformer
from sentence_transformers import SentenceTransformer

# Colab utilities
from google.colab import files

In [22]:
uploaded = files.upload()

Saving anger_encoder.pkl to anger_encoder (1).pkl
Saving anger_model.pkl to anger_model (1).pkl
Saving anxiety_encoder.pkl to anxiety_encoder (1).pkl
Saving anxiety_model.pkl to anxiety_model (1).pkl
Saving confusion_encoder.pkl to confusion_encoder (1).pkl
Saving confusion_model.pkl to confusion_model (1).pkl
Saving emotion_encoder.pkl to emotion_encoder (1).pkl
Saving emotion_model.pkl to emotion_model (1).pkl
Saving guilt_encoder.pkl to guilt_encoder (1).pkl
Saving guilt_model.pkl to guilt_model (1).pkl
Saving hope_encoder.pkl to hope_encoder (1).pkl
Saving hope_model.pkl to hope_model (1).pkl
Saving joy_encoder.pkl to joy_encoder (1).pkl
Saving joy_model.pkl to joy_model (1).pkl
Saving sadness_encoder.pkl to sadness_encoder (1).pkl
Saving sadness_model.pkl to sadness_model (1).pkl


In [23]:
# Initialize mediapipe pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Helper function
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

In [24]:
# Takbir (Raising hands near ears/shoulders)
def check_takbir(image_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.pose_landmarks:
            return False, "No person detected"

        lm = results.pose_landmarks.landmark

        rw = lm[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        lw = lm[mp_pose.PoseLandmark.LEFT_WRIST.value]
        re = lm[mp_pose.PoseLandmark.RIGHT_EAR.value]
        le = lm[mp_pose.PoseLandmark.LEFT_EAR.value]
        rs = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        ls = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

        rw_px = np.array([rw.x * w, rw.y * h])
        lw_px = np.array([lw.x * w, lw.y * h])
        re_px = np.array([re.x * w, re.y * h])
        le_px = np.array([le.x * w, le.y * h])
        rs_px = np.array([rs.x * w, rs.y * h])
        ls_px = np.array([ls.x * w, ls.y * h])

        right_hand_ok = min(rs_px[1], re_px[1]) - 30 < rw_px[1] < max(rs_px[1], re_px[1]) + 30
        left_hand_ok  = min(ls_px[1], le_px[1]) - 30 < lw_px[1] < max(ls_px[1], le_px[1]) + 30

        right_near_head = abs(rw_px[0] - re_px[0]) < 80
        left_near_head  = abs(lw_px[0] - le_px[0]) < 80

        if right_hand_ok and left_hand_ok and right_near_head and left_near_head:
            return True, "Takbir correct."
        else:
            return False, "Raise both hands near ears/shoulders for Takbir."

In [25]:
# Ruku (Bowing)
def check_ruku(image_path):
    image = cv2.imread(image_path)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.pose_landmarks:
        return False, "No person detected"

    lm = results.pose_landmarks.landmark

    left_shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_hip = [lm[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                lm[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

    r_hip, l_hip = lm[mp_pose.PoseLandmark.RIGHT_HIP.value], lm[mp_pose.PoseLandmark.LEFT_HIP.value]
    r_knee, l_knee = lm[mp_pose.PoseLandmark.RIGHT_KNEE.value], lm[mp_pose.PoseLandmark.LEFT_KNEE.value]
    r_ankle, l_ankle = lm[mp_pose.PoseLandmark.RIGHT_ANKLE.value], lm[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    hips_y = (r_hip.y + l_hip.y) / 2
    knees_y = (r_knee.y + l_knee.y) / 2
    ankles_y = (r_ankle.y + l_ankle.y) / 2

    standing = hips_y < knees_y < ankles_y
    if not standing:
        return False, "Stand upright with legs straight before bending into Ruku."

    angle = calculate_angle(left_shoulder, left_hip, left_knee)
    if not (70 <= angle <= 110):
        return False, "Bend your back so it is roughly horizontal in Ruku."

    return True, "Ruku correct: back straight and posture maintained."

In [26]:
# Standing hand placement (hands on chest)
def check_qiyam_hands(image_path):
    image = cv2.imread(image_path)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.pose_landmarks:
        return False, "No person detected"

    lm = results.pose_landmarks.landmark
    rw, lw = lm[mp_pose.PoseLandmark.RIGHT_WRIST], lm[mp_pose.PoseLandmark.LEFT_WRIST]
    rs, ls = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER], lm[mp_pose.PoseLandmark.LEFT_SHOULDER]

    chest_y = (rs.y + ls.y) / 2
    hands_avg_y = (rw.y + lw.y) / 2
    hands_avg_x = (rw.x + lw.x) / 2
    shoulders_avg_x = (rs.x + ls.x) / 2

    if abs(hands_avg_y - chest_y) >= 0.1 or abs(hands_avg_x - shoulders_avg_x) >= 0.05:
        return False, "Place your hands in the center of your chest."

    if rw.y >= lw.y:
        return False, "Place your right hand on top of your left hand."

    r_hip, l_hip = lm[mp_pose.PoseLandmark.RIGHT_HIP], lm[mp_pose.PoseLandmark.LEFT_HIP]
    r_knee, l_knee = lm[mp_pose.PoseLandmark.RIGHT_KNEE], lm[mp_pose.PoseLandmark.LEFT_KNEE]
    r_ankle, l_ankle = lm[mp_pose.PoseLandmark.RIGHT_ANKLE], lm[mp_pose.PoseLandmark.LEFT_ANKLE]

    hips_y = (r_hip.y + l_hip.y) / 2
    knees_y = (r_knee.y + l_knee.y) / 2
    ankles_y = (r_ankle.y + l_ankle.y) / 2

    if not (hips_y < knees_y < ankles_y):
        return False, "Stand upright with your legs straight."

    nose, left_eye, right_eye = lm[mp_pose.PoseLandmark.NOSE], lm[mp_pose.PoseLandmark.LEFT_EYE], lm[mp_pose.PoseLandmark.RIGHT_EYE]
    if ((left_eye.y + right_eye.y) / 2) <= nose.y:
        return False, "Tilt your head downward, looking where you will do sujood."

    return True, "Correct Qiyam posture."

In [27]:
# Sitting between sujood (back view)
def check_sitting_back(image_path):
    image = cv2.imread(image_path)
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.pose_landmarks:
            return False, "No person detected"

        lm = results.pose_landmarks.landmark

        hip_left   = [lm[mp_pose.PoseLandmark.LEFT_HIP.value].x, lm[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee_left  = [lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x, lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle_left = [lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

        hip_right   = [lm[mp_pose.PoseLandmark.RIGHT_HIP.value].x, lm[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        knee_right  = [lm[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, lm[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        ankle_right = [lm[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, lm[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

        left_leg_angle  = calculate_angle(hip_left, knee_left, ankle_left)
        right_leg_angle = calculate_angle(hip_right, knee_right, ankle_right)

        if left_leg_angle >= 100:
            return False, "Fold your left leg more so the knee is closer to the floor."
        if not (70 < right_leg_angle < 130):
            return False, "Adjust your right leg to be upright (around 90 degrees)."

        return True, "Back/side view: Sitting posture correct."

In [28]:
# Sitting between sujood (front view)
def check_sitting_front(image_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.pose_landmarks:
            return False, "No person detected"

        lm = results.pose_landmarks.landmark

        rs = [lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        ls = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        rh = [lm[mp_pose.PoseLandmark.RIGHT_HIP.value].x, lm[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        lh = [lm[mp_pose.PoseLandmark.LEFT_HIP.value].x, lm[mp_pose.PoseLandmark.LEFT_HIP.value].y]

        torso_upright = abs((rs[1]+ls[1])/2 - (rh[1]+lh[1])/2) < 0.15
        if not torso_upright:
            return False, "Keep your torso upright while sitting between sujud."

        wrist_left  = np.array([lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x*w, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y*h])
        wrist_right = np.array([lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x*w, lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y*h])
        knee_left   = np.array([lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x*w, lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y*h])
        knee_right  = np.array([lm[mp_pose.PoseLandmark.RIGHT_KNEE.value].x*w, lm[mp_pose.PoseLandmark.RIGHT_KNEE.value].y*h])

        if np.linalg.norm(wrist_left - knee_left) >= 0.1*h:
            return False, "Place your left hand on your left thigh."
        if np.linalg.norm(wrist_right - knee_right) >= 0.1*h:
            return False, "Place your right hand on your right thigh."

        return True, "Front view: Sitting posture correct."

In [29]:
# Sujood
def check_sujood(image_path):
    image = cv2.imread(image_path)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.pose_landmarks:
        return False, "No person detected"

    lm = results.pose_landmarks.landmark

    left_knee = [lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x, lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    right_knee = [lm[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, lm[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    nose_y = lm[mp_pose.PoseLandmark.NOSE.value].y
    knee_y = (left_knee[1] + right_knee[1]) / 2

    left_elbow, right_elbow = lm[mp_pose.PoseLandmark.LEFT_ELBOW.value], lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    left_wrist, right_wrist = lm[mp_pose.PoseLandmark.LEFT_WRIST.value], lm[mp_pose.PoseLandmark.RIGHT_WRIST.value]

    if not (nose_y > knee_y):
        return False, "Lower your head so it touches the ground in Sujood."
    if not ((left_elbow.y < left_wrist.y - 0.05) and (right_elbow.y < right_wrist.y - 0.05)):
        return False, "Lift your elbows off the ground during Sujood."

    return True, "Sujood correct: head down, elbows lifted."

In [30]:
# Setup
app = Flask(__name__)
CORS(app)

# Load Models
embedder = SentenceTransformer('all-mpnet-base-v2')

emotion_model = joblib.load("emotion_model.pkl")
emotion_encoder = joblib.load("emotion_encoder.pkl")

subcategory_models = {
    "sadness": {"model": joblib.load("sadness_model.pkl"), "encoder": joblib.load("sadness_encoder.pkl")},
    "anxiety": {"model": joblib.load("anxiety_model.pkl"), "encoder": joblib.load("anxiety_encoder.pkl")},
    "anger": {"model": joblib.load("anger_model.pkl"), "encoder": joblib.load("anger_encoder.pkl")},
    "joy": {"model": joblib.load("joy_model.pkl"), "encoder": joblib.load("joy_encoder.pkl")},
    "confusion": {"model": joblib.load("confusion_model.pkl"), "encoder": joblib.load("confusion_encoder.pkl")},
    "hope": {"model": joblib.load("hope_model.pkl"), "encoder": joblib.load("hope_encoder.pkl")},
    "guilt": {"model": joblib.load("guilt_model.pkl"), "encoder": joblib.load("guilt_encoder.pkl")},
}

# Text emotion prediction
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    text = data.get("text", "")

    if not text:
        return jsonify({"error": "No text provided"}), 400

    # Convert input to embedding
    embedding = embedder.encode([text])

    # Predict main emotion
    emotion_pred = emotion_model.predict(embedding)
    emotion = emotion_encoder.inverse_transform(emotion_pred)[0]

    # Subcategory classification
    if emotion in subcategory_models:
        sub_model = subcategory_models[emotion]["model"]
        sub_encoder = subcategory_models[emotion]["encoder"]
        sub_pred = sub_model.predict(embedding)
        subcategory = sub_encoder.inverse_transform(sub_pred)[0]
    else:
        subcategory = "unknown"

    return jsonify({"emotion": emotion, "subcategory": subcategory})

# Image classification
@app.route('/classify', methods=['POST'])
def classify():
    position = request.form.get("position")

    if "image" not in request.files:
        return jsonify({"error": "No image uploaded"}), 400

    image_file = request.files["image"]
    image_path = os.path.join("/content", image_file.filename)
    image_file.save(image_path)

    # Call functions based on position
    if position == "qiyam_hands":
        result, message = check_qiyam_hands(image_path)
    elif position == "ruku":
        result, message = check_ruku(image_path)
    elif position == "sujood":
        result, message = check_sujood(image_path)
    elif position == "sitting_back":
        result, message = check_sitting_back(image_path)
    elif position == "sitting_front":
        result, message = check_sitting_front(image_path)
    elif position == "takbir":
        result, message = check_takbir(image_path)
    else:
        return jsonify({"error": "Unknown position"}), 400

    return jsonify({"result": result, "message": message})


# Run App
if __name__ == "__main__":
    # Just one tunnel for everything
    public_url = ngrok.connect(5000)
    print("API is available at:", public_url)
    print("Use /classify for posture and /predict for emotion")

    app.run(port=5000)

Your API is available at: NgrokTunnel: "https://ba13f6b9023b.ngrok-free.app" -> "http://localhost:5000"
Use /classify for posture and /predict for emotion
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
